In [ ]:
%cd /content
import IPython.display
import os
import sys

if not os.path.exists("/content/sd-inference-server"):
    !git clone https://github.com/arenasys/sd-inference-server.git

%cd /content/sd-inference-server
!git pull

model_folder = "/content/sd-inference-server/models"
try:
    # decline the popup to use the local folder ^
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    model_folder = "/content/drive/My Drive/qDiffusion/models"
    if not os.path.exists(model_folder):
        !mkdir '/content/drive/My Drive/qDiffusion' -p
        !cp -r 'models/' '/content/drive/My Drive/qDiffusion/'
except Exception as e:
    pass

if not os.path.exists("venv"):
    !mkdir venv
    !pip uninstall -q -y tensorflow
    !pip install -q diffusers==0.21.0 k_diffusion==0.0.15 basicsr==1.4.2 lark==1.1.5 transformers==4.29.2 websockets==11.0.3 bson==0.5.10 mega.py==1.0.8 pytorch-lightning==2.0.2 timm==0.9.2 tomesd==0.1.3 pycloudflared==0.2.0 segment-anything==1.0 geffnet==1.0.2 
    IPython.display.clear_output()

    !apt -y update -qq
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
    %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
    IPython.display.clear_output()

if not sys.path[0] == "/content/sd-inference-server/":
    sys.path.insert(0, "/content/sd-inference-server/")

from pycloudflared import try_cloudflare
tunnel_url = try_cloudflare(port=28888, verbose=False)
endpoint_url = tunnel_url.tunnel.replace("https", "wss")

!python remote.py --models "$model_folder" --endpoint "$endpoint_url"

try_cloudflare.terminate(28888)